In [2]:
import pandas as pd
df1 = pd.read_json('/content/dvmanor.json')

df1.head(10)


,reviews
0,"{'id': 1, 'reviewContent': 'I have faced the w..."
1,"{'id': 2, 'reviewContent': 'The checkin and ch..."
2,"{'id': 3, 'reviewContent': 'It's one of the be..."
3,"{'id': 4, 'reviewContent': 'Good rooms for cor..."
4,"{'id': 5, 'reviewContent': 'Good location, cle..."
5,"{'id': 6, 'reviewContent': 'Hotel is located i..."
6,"{'id': 7, 'reviewContent': 'Excellent locality..."
7,"{'id': 8, 'reviewContent': 'Nice regular stay,..."
8,"{'id': 9, 'reviewContent': 'The stay is very c..."
9,"{'id': 10, 'reviewContent': 'Located in centre..."


In [3]:
df= pd.DataFrame(df1['reviews'].tolist())
df

,id,reviewContent
0,1,I have faced the worst experience in this hote...
1,2,The checkin and checkout was professionally ha...
2,3,It's one of the best hotels i have ever stayed...
3,4,"Good rooms for corporate stay, food was ok as ..."
4,5,"Good location, clean rooms, toilets/bathrooms/..."
...,...,...
665,666,Good Hotel with excellent ambience and facilit...
666,667,"A nice hotel with good ambience, food and serv..."
667,668,we are happy. and will stay again if we need t...
668,669,This is a nice hotel to stay.


In [4]:
df.head(6)

,id,reviewContent
0,1,I have faced the worst experience in this hote...
1,2,The checkin and checkout was professionally ha...
2,3,It's one of the best hotels i have ever stayed...
3,4,"Good rooms for corporate stay, food was ok as ..."
4,5,"Good location, clean rooms, toilets/bathrooms/..."
5,6,Hotel is located ideally and food is delicious...


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [6]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
df

,id,reviewContent
0,1,I have faced the worst experience in this hote...
1,2,The checkin and checkout was professionally ha...
2,3,It's one of the best hotels i have ever stayed...
3,4,"Good rooms for corporate stay, food was ok as ..."
4,5,"Good location, clean rooms, toilets/bathrooms/..."
...,...,...
665,666,Good Hotel with excellent ambience and facilit...
666,667,"A nice hotel with good ambience, food and serv..."
667,668,we are happy. and will stay again if we need t...
668,669,This is a nice hotel to stay.


In [8]:
dfsplit = pd.DataFrame(df)

# Function to split text into sentences
def split_sentences(text):
    sentences = re.split(r'[.!,]| and | but | or ', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Apply sentence splitting
dfsplit['sentences'] = df['reviewContent'].apply(split_sentences)

# Explode the sentences list into separate rows
dfSplit1 = dfsplit.explode('sentences').reset_index(drop=True)

# Display DataFrame with sentences split at full stops, commas, and conjunctions
print(dfSplit1)

       id                                      reviewContent  \
0       1  I have faced the worst experience in this hote...   
1       1  I have faced the worst experience in this hote...   
2       2  The checkin and checkout was professionally ha...   
3       2  The checkin and checkout was professionally ha...   
4       2  The checkin and checkout was professionally ha...   
...   ...                                                ...   
3431  668  we are happy. and will stay again if we need t...   
3432  669                      This is a nice hotel to stay.   
3433  670  The hotel provides world-class service, nice s...   
3434  670  The hotel provides world-class service, nice s...   
3435  670  The hotel provides world-class service, nice s...   

                                            sentences  
0     I have faced the worst experience in this hotel  
1          Very un professional behavior by the staff  
2                                         The checkin  
3      

In [9]:
dfSplit1.to_csv('dfsplit1.csv', index=False)
dfSplit1

,id,reviewContent,sentences
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel
1,1,I have faced the worst experience in this hote...,Very un professional behavior by the staff
2,2,The checkin and checkout was professionally ha...,The checkin
3,2,The checkin and checkout was professionally ha...,checkout was professionally handled
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team
...,...,...,...
3431,668,we are happy. and will stay again if we need t...,good
3432,669,This is a nice hotel to stay.,This is a nice hotel to stay
3433,670,"The hotel provides world-class service, nice s...",The hotel provides world-class service
3434,670,"The hotel provides world-class service, nice s...",nice staff


In [10]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')



dfsr1 = pd.DataFrame(dfSplit1)

# Function to preprocess text
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Apply preprocessing to the 'reviewContent' column
dfSplit1['preprocessed_text'] = dfSplit1['sentences'].apply(preprocess_text)

# Function to get sentiment polarity
def get_sentiment(sentence):
    return TextBlob(sentence).sentiment.polarity

# Apply the function to the 'preprocessed_text' column
dfSplit1['sentiment_score'] = dfSplit1['preprocessed_text'].apply(get_sentiment)

# Display DataFrame with sentiment scores
print(dfSplit1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


       id                                      reviewContent  \
0       1  I have faced the worst experience in this hote...   
1       1  I have faced the worst experience in this hote...   
2       2  The checkin and checkout was professionally ha...   
3       2  The checkin and checkout was professionally ha...   
4       2  The checkin and checkout was professionally ha...   
...   ...                                                ...   
3431  668  we are happy. and will stay again if we need t...   
3432  669                      This is a nice hotel to stay.   
3433  670  The hotel provides world-class service, nice s...   
3434  670  The hotel provides world-class service, nice s...   
3435  670  The hotel provides world-class service, nice s...   

                                            sentences  \
0     I have faced the worst experience in this hotel   
1          Very un professional behavior by the staff   
2                                         The checkin   
3  

In [11]:
dfSplit1

,id,reviewContent,sentences,preprocessed_text,sentiment_score
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel,faced worst experience hotel,-1.0
1,1,I have faced the worst experience in this hote...,Very un professional behavior by the staff,un professional behavior staff,0.1
2,2,The checkin and checkout was professionally ha...,The checkin,checkin,0.0
3,2,The checkin and checkout was professionally ha...,checkout was professionally handled,checkout professionally handled,0.1
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team,spring restaurant good team,0.7
...,...,...,...,...,...
3431,668,we are happy. and will stay again if we need t...,good,good,0.7
3432,669,This is a nice hotel to stay.,This is a nice hotel to stay,nice hotel stay,0.6
3433,670,"The hotel provides world-class service, nice s...",The hotel provides world-class service,hotel provides world-class service,0.0
3434,670,"The hotel provides world-class service, nice s...",nice staff,nice staff,0.6


In [12]:
def get_sentiment_category(score):
    if score > 0.1:
        return 'positive'
    elif score < -0.1:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to the 'sentiment_score' column
dfSplit1['sentiment_category'] = dfSplit1['sentiment_score'].apply(get_sentiment_category)

# Display DataFrame with sentiment categories
dfSplit1

,id,reviewContent,sentences,preprocessed_text,sentiment_score,sentiment_category
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel,faced worst experience hotel,-1.0,negative
1,1,I have faced the worst experience in this hote...,Very un professional behavior by the staff,un professional behavior staff,0.1,neutral
2,2,The checkin and checkout was professionally ha...,The checkin,checkin,0.0,neutral
3,2,The checkin and checkout was professionally ha...,checkout was professionally handled,checkout professionally handled,0.1,neutral
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team,spring restaurant good team,0.7,positive
...,...,...,...,...,...,...
3431,668,we are happy. and will stay again if we need t...,good,good,0.7,positive
3432,669,This is a nice hotel to stay.,This is a nice hotel to stay,nice hotel stay,0.6,positive
3433,670,"The hotel provides world-class service, nice s...",The hotel provides world-class service,hotel provides world-class service,0.0,neutral
3434,670,"The hotel provides world-class service, nice s...",nice staff,nice staff,0.6,positive


In [13]:
from collections import Counter

# Tokenize preprocessed text and count occurrences of each token
tokens = ' '.join(dfSplit1['preprocessed_text']).split()
word_counts = Counter(tokens)

# Get the most common keywords
most_common_keywords = word_counts.most_common(10)

print("Most common keywords in preprocessed text:")
for keyword, count in most_common_keywords:
    print(f"{keyword}: {count} times")

Most common keywords in preprocessed text:
good: 558 times
hotel: 418 times
room: 331 times
food: 262 times
service: 223 times
stay: 210 times
staff: 194 times
location: 146 times
breakfast: 132 times
nice: 119 times


In [14]:
# Create a DataFrame containing only rows where sentiment_category is "negative"
negative_reviews_df = dfSplit1[dfSplit1['sentiment_category'] == 'negative']

# Display the new DataFrame
negative_reviews_df



,id,reviewContent,sentences,preprocessed_text,sentiment_score,sentiment_category
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel,faced worst experience hotel,-1.00000,negative
88,16,"Room was neat and tidy and hotel as well, staf...",staff were humble,staff humble,-0.20000,negative
131,19,We have pre-booked a room at DV Manor from 23r...,A pathetic experience,pathetic experience,-1.00000,negative
133,19,We have pre-booked a room at DV Manor from 23r...,Unless management takes care of these small is...,unless management take care small issue,-0.25000,negative
146,22,Quality of bed and bed sheets are poor with th...,bed sheets are poor with the price i paid,bed sheet poor price paid,-0.40000,negative
...,...,...,...,...,...,...
3293,650,Excited for Vijayawada trip for the first time...,as per my experience the room was crap with to...,per experience room crap torn cardboard,-0.80000,negative
3334,655,"Overall, it's a good hotel with great location...",Wi-Fi was bad,wi-fi bad,-0.70000,negative
3335,655,"Overall, it's a good hotel with great location...",Mattress was too fluffy for comfort,mattress fluffy comfort,-0.20000,negative
3352,657,"Awesome hotel and good food. Clean rooms, good...",swimming pool is missing,swimming pool missing,-0.20000,negative


In [15]:
from collections import Counter

# Tokenize preprocessed text and count occurrences of each token
tokens = ' '.join(negative_reviews_df['preprocessed_text']).split()
word_counts = Counter(tokens)

# Get the most common keywords
most_common_keywords = word_counts.most_common(10)

print("Most common keywords in preprocessed text:")
for keyword, count in most_common_keywords:
    print(f"{keyword}: {count} times")

Most common keywords in preprocessed text:
room: 27 times
poor: 25 times
service: 25 times
hotel: 24 times
bad: 21 times
worst: 16 times
little: 16 times
experience: 15 times
expensive: 15 times
small: 13 times


In [16]:
# Create a DataFrame containing only rows where sentiment_category is "negative"
positive_reviews_df = dfSplit1[dfSplit1['sentiment_category'] == 'positive']

# Display the new DataFrame
positive_reviews_df

,id,reviewContent,sentences,preprocessed_text,sentiment_score,sentiment_category
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team,spring restaurant good team,0.700000,positive
5,2,The checkin and checkout was professionally ha...,The rooms were clean,room clean,0.366667,positive
7,2,The checkin and checkout was professionally ha...,aircon was overly super cool,aircon overly super cool,0.341667,positive
10,3,It's one of the best hotels i have ever stayed...,It's one of the best hotels i have ever stayed in,'s one best hotel ever stayed,1.000000,positive
11,3,It's one of the best hotels i have ever stayed...,D v manor is the best in all aspects - best lo...,v manor best aspect - best location,1.000000,positive
...,...,...,...,...,...,...
3429,668,we are happy. and will stay again if we need t...,we are happy,happy,0.800000,positive
3431,668,we are happy. and will stay again if we need t...,good,good,0.700000,positive
3432,669,This is a nice hotel to stay.,This is a nice hotel to stay,nice hotel stay,0.600000,positive
3434,670,"The hotel provides world-class service, nice s...",nice staff,nice staff,0.600000,positive


In [17]:
from collections import Counter

# Tokenize preprocessed text and count occurrences of each token
tokens = ' '.join(positive_reviews_df['preprocessed_text']).split()
word_counts = Counter(tokens)

# Get the most common keywords
most_common_keywords = word_counts.most_common(10)

print("Most common keywords in preprocessed text:")
for keyword, count in most_common_keywords:
    print(f"{keyword}: {count} times")

Most common keywords in preprocessed text:
good: 557 times
hotel: 233 times
food: 190 times
stay: 171 times
room: 155 times
service: 131 times
nice: 119 times
excellent: 110 times
staff: 100 times
location: 93 times


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Function to generate n-grams
def generate_ngrams(text, n=2):
    tokens = text.split()
    if len(tokens) < n:
        return []
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in ngrams]

# Apply n-gram generation and count the occurrences of each n-gram
ngram_counts = Counter([ngram for text in dfSplit1['preprocessed_text'] for ngram in generate_ngrams(text)])

# Get the most common repeating n-grams
most_common_repeating_ngrams = [ngram for ngram, count in ngram_counts.items() if count > 1]

# Display repeating n-grams
print("Repeating sequences of words (n-grams):")
for ngram in most_common_repeating_ngrams:
    print(ngram)


Repeating sequences of words (n-grams):
worst experience
experience hotel
restaurant good
room clean
's one
one best
best hotel
v manor
food option
great hospitality
good room
food ok
ok per
good location
clean room
excellent food
room service
service quick
hotel located
food delicious
staff behaviour
delicious food
service good
regular stay
comfortable room
stay comfortable
food quality
quality good
best place
located centre
centre city
good quality
quality excellent
break fast
house keeping
taste food
service fast
staying hotel
hotel next
next visit
dv manor
quality food
food outstanding
helping staff
guest breakfast
live counter
value money
great stay
excellent hotel
spacious room
food good
always stay
stay hotel
visit vijayawada
great hotel
hotel good
good amenity
complimentary breakfast
breakfast awesome
got room
room booked
better price
make trip
top floor
room neat
hotel well
breakfast buffet
restaurant staff
staff also
also polite
also good
food also
maintain cleanness
treatmen

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Function to generate n-grams
def generate_ngrams(text, n=3):
    tokens = text.split()
    if len(tokens) < n:
        return []
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in ngrams]

# Apply n-gram generation and count the occurrences of each n-gram
ngram_counts = Counter([ngram for text in dfSplit1['preprocessed_text'] for ngram in generate_ngrams(text)])

# Get the most common repeating n-grams
most_common_repeating_ngrams = [ngram for ngram, count in ngram_counts.items() if count > 1]

# Display repeating n-grams
print("Repeating sequences of words (n-grams):")
for ngram in most_common_repeating_ngrams:
    print(ngram)


Repeating sequences of words (n-grams):
one best hotel
food quality good
hotel next visit
always stay hotel
hotel good amenity
staff also polite
food also good
person booked room
worth money spent
really enjoyed stay
staff also good
situated heart city
good room service
improve room service
nice place stay
good place stay
place stay family
room quality good
good staff member
house keeping staff
value added service
need improve service
good quality food
wonderful experience staying
hotel located prime
located prime location
prime location vijaywada
one best option
location hotel good
staff wearing mask
room size good
like 5 star
best hotel stay
good hotel stay
hotel stay vijayawada
take lot time
quality dv manor
king size bed
good food quality
come back hotel
located mg road
excellent food quality
well maintained hotel
nice stay hotel
thanks make trip
stayed 2 night
would definitely recommend
book hotel next
< br /
br / >
back dv manor
term quality service
great value money
effort towar

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Function to generate n-grams
def generate_ngrams(text, n=4):
    tokens = text.split()
    if len(tokens) < n:
        return []
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in ngrams]

# Apply n-gram generation and count the occurrences of each n-gram
ngram_counts = Counter([ngram for text in dfSplit1['preprocessed_text'] for ngram in generate_ngrams(text)])

# Get the most common repeating n-grams
most_common_repeating_ngrams = [ngram for ngram, count in ngram_counts.items() if count > 1]

# Display repeating n-grams
print("Repeating sequences of words (n-grams):")
for ngram in most_common_repeating_ngrams:
    print(ngram)


Repeating sequences of words (n-grams):
hotel located prime location
located prime location vijaywada
< br / >
close main shopping area
br / > experience
/ > experience good
br / > good
br / > comfortable
br / > room
br / > liked
well behaved professional staff
hotel located heart city
room ] service excellent
food quality good especially
quality good especially south
good especially south indian
especially south indian food
nice stay hotel mg
stay hotel mg road
room service little slow
breakfast part deal made
part deal made special
seem know regular visitor
good stay business trip
lot disturbance room next
disturbance room next conference
room next conference hall
buffet breakfast spread limited
overall experience hotel good
dv manor vijayawada one
found cockroach running bed
cockroach running bed 's
running bed 's worst
bed 's worst thing
's worst thing hotel
imagine many star give
many star give hotel
could get device function
place clear compared india
room updated given price
upd

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Function to generate n-grams
def generate_ngrams(text, n=4):
    tokens = text.split()
    if len(tokens) < n:
        return []
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in ngrams]

# Apply n-gram generation and count the occurrences of each n-gram
ngram_counts = Counter([ngram for text in dfSplit1['preprocessed_text'] for ngram in generate_ngrams(text)])

# Get the most common repeating n-grams
most_common_repeating_ngrams = [ngram for ngram, count in ngram_counts.items() if count > 1]

# Display repeating n-grams
print("Repeating sequences of words (n-grams):")
for ngram in most_common_repeating_ngrams:
    print(ngram)


Repeating sequences of words (n-grams):
hotel located prime location
located prime location vijaywada
< br / >
close main shopping area
br / > experience
/ > experience good
br / > good
br / > comfortable
br / > room
br / > liked
well behaved professional staff
hotel located heart city
room ] service excellent
food quality good especially
quality good especially south
good especially south indian
especially south indian food
nice stay hotel mg
stay hotel mg road
room service little slow
breakfast part deal made
part deal made special
seem know regular visitor
good stay business trip
lot disturbance room next
disturbance room next conference
room next conference hall
buffet breakfast spread limited
overall experience hotel good
dv manor vijayawada one
found cockroach running bed
cockroach running bed 's
running bed 's worst
bed 's worst thing
's worst thing hotel
imagine many star give
many star give hotel
could get device function
place clear compared india
room updated given price
upd

In [ ]:
#take scentence, all scentences - break, extract keywords per scentence.
#3 keywords - take a key word from sentence 1 nd other sentence 2 and combine 1 and 2 keywords repeating the mostt.
#repetitive adjectives and nouns can be considered inorder to check.
#joining the kw to kw and checking for combination of that with other sentence. 4. extract DIRTY , ROOM - addon by checking the PnC.
#kw- kw , n-adj , adj-adj combo.
#once try n-gram thing.
#how chatbots used in multiple industries ,evolution of chatbots.
#ten case studies on chatbots.

In [24]:
dfSplit1.head(5)

,id,reviewContent,sentences,preprocessed_text,sentiment_score,sentiment_category
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel,faced worst experience hotel,-1.0,negative
1,1,I have faced the worst experience in this hote...,Very un professional behavior by the staff,un professional behavior staff,0.1,neutral
2,2,The checkin and checkout was professionally ha...,The checkin,checkin,0.0,neutral
3,2,The checkin and checkout was professionally ha...,checkout was professionally handled,checkout professionally handled,0.1,neutral
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team,spring restaurant good team,0.7,positive


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Function to extract keywords from preprocessed text
def extract_keywords(text, n=5):
    # Tokenize text
    tokens = text.split()

    # Count the occurrences of each word
    word_counts = Counter(tokens)

    # Get the most common words
    keywords = [word for word, count in word_counts.most_common(n)]

    return keywords

# Extract keywords and append them to the DataFrame
dfSplit1['keywords'] = dfSplit1['preprocessed_text'].apply(extract_keywords)

# Display DataFrame with keywords
print(dfSplit1)


       id                                      reviewContent  \
0       1  I have faced the worst experience in this hote...   
1       1  I have faced the worst experience in this hote...   
2       2  The checkin and checkout was professionally ha...   
3       2  The checkin and checkout was professionally ha...   
4       2  The checkin and checkout was professionally ha...   
...   ...                                                ...   
3431  668  we are happy. and will stay again if we need t...   
3432  669                      This is a nice hotel to stay.   
3433  670  The hotel provides world-class service, nice s...   
3434  670  The hotel provides world-class service, nice s...   
3435  670  The hotel provides world-class service, nice s...   

                                            sentences  \
0     I have faced the worst experience in this hotel   
1          Very un professional behavior by the staff   
2                                         The checkin   
3  

In [26]:
dfSplit1


,id,reviewContent,sentences,preprocessed_text,sentiment_score,sentiment_category,keywords
0,1,I have faced the worst experience in this hote...,I have faced the worst experience in this hotel,faced worst experience hotel,-1.0,negative,"[faced, worst, experience, hotel]"
1,1,I have faced the worst experience in this hote...,Very un professional behavior by the staff,un professional behavior staff,0.1,neutral,"[un, professional, behavior, staff]"
2,2,The checkin and checkout was professionally ha...,The checkin,checkin,0.0,neutral,[checkin]
3,2,The checkin and checkout was professionally ha...,checkout was professionally handled,checkout professionally handled,0.1,neutral,"[checkout, professionally, handled]"
4,2,The checkin and checkout was professionally ha...,The springs restaurant was good as was the team,spring restaurant good team,0.7,positive,"[spring, restaurant, good, team]"
...,...,...,...,...,...,...,...
3431,668,we are happy. and will stay again if we need t...,good,good,0.7,positive,[good]
3432,669,This is a nice hotel to stay.,This is a nice hotel to stay,nice hotel stay,0.6,positive,"[nice, hotel, stay]"
3433,670,"The hotel provides world-class service, nice s...",The hotel provides world-class service,hotel provides world-class service,0.0,neutral,"[hotel, provides, world-class, service]"
3434,670,"The hotel provides world-class service, nice s...",nice staff,nice staff,0.6,positive,"[nice, staff]"


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import combinations

# Function to extract n-grams from preprocessed text
def extract_ngrams(text, n=2):
    # Tokenize text
    tokens = text.split()

    # Generate n-grams
    ngrams = zip(*[tokens[i:] for i in range(n)])
    ngrams = [' '.join(ngram) for ngram in ngrams]

    return ngrams

# Function to extract keywords and check if combination exists in preprocessed text
def extract_and_check_keywords(text, keyword1, keyword2, n=2):
    # Extract n-grams from preprocessed text
    ngrams = extract_ngrams(text, n)

    # Check if combination of keywords exists in n-grams
    if any(keyword1 in ngram and keyword2 in ngram for ngram in ngrams):
        return 1
    else:
        return 0

# Choose two keywords
keyword1 = 'worst'
keyword2 = 'service'

# Extract keywords and check if combination exists in preprocessed text
dfSplit1['has_keywords'] = dfSplit1['preprocessed_text'].apply(
    lambda x: extract_and_check_keywords(x,keyword1, keyword2))

# Display DataFrame with 'has_keywords' column
print(dfSplit1)


       id                                      reviewContent  \
0       1  I have faced the worst experience in this hote...   
1       1  I have faced the worst experience in this hote...   
2       2  The checkin and checkout was professionally ha...   
3       2  The checkin and checkout was professionally ha...   
4       2  The checkin and checkout was professionally ha...   
...   ...                                                ...   
3431  668  we are happy. and will stay again if we need t...   
3432  669                      This is a nice hotel to stay.   
3433  670  The hotel provides world-class service, nice s...   
3434  670  The hotel provides world-class service, nice s...   
3435  670  The hotel provides world-class service, nice s...   

                                            sentences  \
0     I have faced the worst experience in this hotel   
1          Very un professional behavior by the staff   
2                                         The checkin   
3  

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import combinations

# Function to extract n-grams from preprocessed text
def extract_ngrams(text, n=2):
    # Tokenize text
    tokens = text.split()

    # Generate n-grams
    ngrams = zip(*[tokens[i:] for i in range(n)])
    ngrams = [' '.join(ngram) for ngram in ngrams]

    return ngrams

# Function to check if combination of keywords exists in preprocessed text
def check_keywords_combination(text, keywords):
    # Extract n-grams from preprocessed text
    ngrams = extract_ngrams(text)

    # Check if combination of keywords exists in n-grams
    if all(keyword in ' '.join(ngrams) for keyword in keywords):
        return 1
    else:
        return 0

# Iterate over different combinations of keywords
for keyword1, keyword2 in combinations(['good', 'service', 'location', 'clean', 'food'], 2):
    # Create a column name based on keywords
    column_name = f'{keyword1}_{keyword2}_exists'

    # Check if combination of keywords exists in preprocessed text
    dfSplit1[column_name] = dfSplit1['preprocessed_text'].apply(
        lambda x: check_keywords_combination(x, [keyword1, keyword2]))

# Display DataFrame with new columns
print(dfSplit1)


       id                                      reviewContent  \
0       1  I have faced the worst experience in this hote...   
1       1  I have faced the worst experience in this hote...   
2       2  The checkin and checkout was professionally ha...   
3       2  The checkin and checkout was professionally ha...   
4       2  The checkin and checkout was professionally ha...   
...   ...                                                ...   
3431  668  we are happy. and will stay again if we need t...   
3432  669                      This is a nice hotel to stay.   
3433  670  The hotel provides world-class service, nice s...   
3434  670  The hotel provides world-class service, nice s...   
3435  670  The hotel provides world-class service, nice s...   

                                            sentences  \
0     I have faced the worst experience in this hotel   
1          Very un professional behavior by the staff   
2                                         The checkin   
3  

In [33]:
# Print data where has_keywords = 1
print(dfSplit1[dfSplit1['has_keywords'] == 1])


       id                                      reviewContent  \
486   120  The service they gave is the worst service i e...   
2067  490  Worst service and I had never experienced in m...   

                                              sentences  \
486   The service they gave is the worst service i e...   
2067                                      Worst service   

                                      preprocessed_text  sentiment_score  \
486   service gave worst service ever highly recomme...          -0.1475   
2067                                      worst service          -1.0000   

     sentiment_category                             keywords  has_keywords  \
486            negative  [service, hotel, gave, worst, ever]             1   
2067           negative                     [worst, service]             1   

      good_service_exists  good_location_exists  ...  service_location_exists  \
486                     0                     0  ...                        0   
206